# <p style="font-family:newtimeroman; text-align:center; font-size:40px;">Coleridge Initiative - Show US the Data<br>Discover how data is used for the public good</p>


<a id='0'></a>
# <p style="background-color:purple; font-family:newtimeroman; font-size:100%; text-align:center; border-radius: 15px 50px; color:white;">Please dont forget to upvote the notebook </p>

## In this notebook basically we have to predict text for some strings by using nlp techniques
*** 
>The objective of the competition is to identify the mention of datasets within scientific publications.

This competition challenges data scientists to show how publicly funded data are used to serve science and society. Evidence through data is critical if government is to address the many threats facing society, including; pandemics, climate change, Alzheimer’s disease, child hunger, increasing food production, maintaining biodiversity, and addressing many other challenges. Yet much of the information about data necessary to inform evidence and science is locked inside publications.

Can natural language processing find the hidden-in-plain-sight data citations? Can machine learning find the link between the words used in research articles and the data referenced in the article?

Now is the time for data scientists to help restore trust in data and evidence. In the United States, federal agencies are now mandated to show how their data are being used. The new Foundations of Evidence-based Policymaking Act requires agencies to modernize their data management. New Presidential Executive Orders are pushing government agencies to make evidence-based decisions based on the best available data and science. And the government is working to respond in an open and transparent way.

This competition will build just such an open and transparent approach. The results will show how public data are being used in science and help the government make wiser, more transparent public investments. It will help move researchers and governments from using ad-hoc methods to automated ways of finding out what datasets are being used to solve problems, what measures are being generated, and which researchers are the experts. Previous competitions have shown that it is possible to develop algorithms to automate the search and discovery of references to data. Now, we want the Kaggle community to develop the best approaches to identify critical datasets used in scientific publications.

In this competition, you'll use natural language processing (NLP) to automate the discovery of how scientific data are referenced in publications. Utilizing the full text of scientific publications from numerous research areas gathered from CHORUS publisher members and other sources, you'll identify data sets that the publications' authors used in their work.

It achieves this goal by working with the agencies to create value for the taxpayer from the careful use of data by building new technologies to enable secure access to and sharing of confidential microdata and by training agency staff to acquire modern data skills.

If successful, you'll help support evidence in government data. Automated NLP approaches will enable government agencies and researchers to quickly find the information they need. The approach will be used to develop data usage scorecards to better enable agencies to show how their data are used and bring down a critical barrier to the access and use of public data.

<a id='1'></a>
# <p style="background-color:blue; font-family:newtimeroman; font-size:120%; text-align:center; border-radius: 10px 25px;color:white;">Table of Content</p>
* [1. Importing Modules & Libraries](#1)
* [2. Data Exploration](#2)
* [3. Data Vizualization](#3)
* [4. Data Cleaning](#4)
* [5. Baseline Model & Submission](#5)

<a id='1'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">1. Importing Modules & Libraries</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px
import plotly.graph_objects as go

import nltk
import urllib.request
from PIL import Image
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.max_length = 4000000
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string

%matplotlib inline

os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')

We are provided with 4 main pieces of data:

* `train.csv:` The CSV file containing all the metadata of the publications, such as their title and the dataset they utilize.
* `train:` The directory containing the actual publications that are referenced in train.csvin JSON format.
* `test:` The directory containing the actual publications that will be used for testing purposes (thus, with no ground truth CSV file available).
* `sample_submission.csv:` The CSV file containing all the publications IDs in the test set, for which we'll have to populate the prediction column.

<a id='2'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">2. Data Exploration</p>

# Reading csv files and train & test file paths

In [ ]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

<a id='1'></a>
## <p style="text-align:center;">Data Description</p>
train.csv - labels and metadata for the training set train/test directory - the full text of the training/test set's publications in JSON format, broken into sections with section titles

* `id` - publication id - note that there are multiple rows for some training documents, indicating multiple mentioned datasets.
* `pub_title` - title of the publication (a small number of publications have the same title).
* `dataset_title` - the title of the dataset that is mentioned within the publication.
* `dataset_label` - a portion of the text that indicates the dataset.
* `cleaned_label` - the dataset_label, as passed through the clean_text function from the Evaluation page.

sample_submission.csv - a sample submission file in the correct format.
* `Id` - publication id.
* `PredictionString` - To be filled with equivalent of cleaned_label of train data.

#  Knowledge Bank : Literal Matching

# Number of Unique values in each column

<p style="color:blue;font-size:20px;">📝Great! we don't have any null values.</p>

### Observations

- 1) There are duplicate id's meaning that there are some pulications that are using mutiple datasets. That's why that id is repeating.
- 2) Same is the case with pub_title. A single publication is using mutiple datasets.
- 3) There is NO one to one mapping of id and pub_title. Meaning that there are cases when two different publications (from two different authors) have same title. Well, interesting!!!
- 4) There 45 dataset titles but 130 dataet labels. Meaning that there are some datasets that has multiple labels. We'll look into how these two are related.

# Examining the section titles in each json file. Some json files have different titles, few are common in some

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# No. of different dataset_label according to dataset_title

# Duplicate ID,pub_title but different datasets

<p style="color: blue;font-size:15px;">📝 We have duplicate ids but we are not going to remove them from data because one research paper may consist of more than one datasets. Anyhow duplicate ids with respective datasets are shown below</p>

# Single publication may have multiple datasets

<a id='3'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">3. Data Vizualization</p>

# Distribution of Id, pub_title in training data. How frequently each value occurs

#  Distribution of pub_title in data (first 20)

# Distribution of dataset_title in data

# Distribution of dataset_label in data

# Relationship b/w dataset_title and dataset_label

# Same Visualization on scatter plot

# Grouping dataset of title and label

<p style="color:blue;font-size:25px;">📝Merge all text from json into train.csv with colmn name text</p>

In [ ]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

<p style="color:blue;font-size:20px;">📝tqdm is used to show any code running with a progress bar</p>

In [ ]:
%%time
tqdm.pandas()   
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

<p style="color:blue;font-size:20px;">📝We have our text appended in our train dataframe</p>

In [ ]:
train_df.head()

In [ ]:
%%time
tqdm.pandas()
sample_sub['text'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path))

<p style="color:blue;font-size:20px;">📝Also, we have the text of for the sample_submission file</p>

In [ ]:
sample_sub.head()

In [ ]:
sample_sub['text']

<a id='4'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">4. Data Cleaning</p>

In [ ]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('r[^\w\s]', ' ', str(text).lower()).strip()
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    text = lem.lemmatize(text)
#     text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    
    return text

In [ ]:
%%time
tqdm.pandas()
train_df['text'] = train_df['text'].progress_apply(text_cleaning)

<p style="color:blue;font-size:20px;">📝We have our data cleaned!</p>

In [ ]:
train_df.head()

In [ ]:
text = train_df['text']
print(text)

<p style="color:blue;font-size:20px;">📝Remove stop words from <b>cleaned_label</b> of train.csv</p>

In [ ]:
words =list(train_df['cleaned_label'].values)
stopwords=['ourselves', 'hers','the','of','and','in', 'between', 'yourself', 'but', 'again','of', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopwords:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist

<p style="color:blue;font-size:20px;">📝100 Most common words (cleaned_label) - WordCloud</p>

<p style="color:blue;font-size:20px;">📝 25 Most Common Words in cleaned_label</p>

<p style="color:blue;font-size:20px;">📝Remove all special characters, punctuation and spaces from string
</p>

*Command to do a regex substitution*

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

<a id='5'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">5. Baseline model and Submission</p>

In [ ]:
temp_1 = [x.lower() for x in train_df['dataset_label'].unique()]
temp_2 = [x.lower() for x in train_df['dataset_title'].unique()]
temp_3 = [x.lower() for x in train_df['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)
id_list = []
lables_list = []
for index, row in tqdm(sample_sub.iterrows()):
    sample_text = row['text']
    row_id = row['Id']
    temp_df = train_df[train_df['text'] == text_cleaning(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    for known_label in existing_labels:
        if known_label in sample_text.lower():
            cleaned_labels.append(clean_text(known_label))
    cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

In [ ]:
submission1 = pd.DataFrame()
submission1['Id'] = id_list
submission1['PredictionString'] = lables_list

In [ ]:
submission1.head()

# # # from this

In [ ]:
from spacy import displacy
from collections import Counter
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
sample_sub2 = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

In [ ]:
%%time
tqdm.pandas()
sample_sub2['text'] = sample_sub2['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path))

In [ ]:
sample_sub2['text']

In [ ]:
#del train_df
del sample_sub

In [ ]:
sample_sub2.head()

In [ ]:
import gc
gc.collect()

In [ ]:
datalist = pd.read_csv('../input/para-data/datalist.csv')

In [ ]:
datalist.head()

In [ ]:
datasetLabels= datalist['Data']

 # extract punctuations/ short forms

In [ ]:
import re
lbl_list1=re.findall('\\b[A-Z][A-Z.&]{2,7}\\b', str(datasetLabels))
print(lbl_list1)

two punctuation like (NCMP FFE)

In [ ]:
# from string import *
# lbl_list2 = [word for word in datasetLabels if all([letter in punctuation+ascii_uppercase+digits+' ' for letter in word]) and word]
# print(lbl_list2)

# **SPACY**

In [ ]:
sub_token = ''
for i in datasetLabels:
   sub_token += str(i)

In [ ]:
doc = nlp(str(sub_token))
print([(X.text, X.label_) for X in doc.ents])

In [ ]:
labels = [x.label_ for x in doc.ents]
Counter(labels)

In [ ]:
items = [x.text for x in doc.ents]
Counter(items).most_common()

In [ ]:
sentences = [x for x in doc.sents]
#print(sentences[2390])



# Extract only ORG

In [ ]:
dict([(str(x), x.label_) for x in nlp(str(sentences)).ents
       if x.label_ == 'ORG'
           ])


In [ ]:
items = [x.text for x in doc.ents
         if x.label_ == 'ORG']
Counter(items).most_common()

In [ ]:
dbLabels = [x.text for x in doc.ents
         if x.label_ == 'ORG']
print(dbLabels)

In [ ]:
dbLabels = [i.replace("'", '') for i in dbLabels]
print(dbLabels)

In [ ]:
# dbLabels.remove('Education')
# print(dbLabels)

**MERGE LISTS**

In [ ]:
fnl_lbl_list= lbl_list1 + dbLabels
print(fnl_lbl_list)

In [ ]:
import collections
fnl_lbl_list=[item for item, count in collections.Counter(fnl_lbl_list).items()]
print(fnl_lbl_list)

In [ ]:
#DBLables=str(DBLables).replace("the ", "")
#print(DBLables)

In [ ]:
# Python3 code to demonstrate
# Remove words containing list characters
# using list comprehension + all()
from itertools import groupby

# initializing list
#test_list = ['gfg', 'is', 'best', 'for', 'geeks']

# initializing char list
char_list = ['\\','et al', 'Table','≈','~','http:','www.','.com','%','e.g','2003','2009',
             '[CLS]','LI','GCTA','SSGAC','NPS','SEP','CLS',
             'STATA','CEU','MDS','CAHA','DEM','SLR','LAS',
             'EFNSE NE','EFNSE','CHARGE','COGENT'
             
            ]            

# printing original list

# printing character list
print ("The character list is : " + str(char_list))

# Remove words containing list characters
# using list comprehension + all()
fnl_lbl_list = [ele for ele in fnl_lbl_list if all(ch not in ele for ch in char_list)]

# printing result
print ("The filtered strings are : " + str(fnl_lbl_list))

In [ ]:
#str(DBLables.isalpha() 
#DBLables=[element for element in DBLables if all(digit not in element for digit in "1234567890")]
#print(DBLables)

In [ ]:
a = [(x,i) for x, y in zip(sample_sub2['Id'],sample_sub2['text']) for i in fnl_lbl_list if i in y]
print (a)

In [ ]:
sample_sub2 = pd.DataFrame(a, columns=['Id', 'PredictionString'])
sample_sub2 = sample_sub2.groupby('Id').agg({'PredictionString': '|'.join}).reset_index()
sample_sub2.head()

In [ ]:
sample_sub2['PredictionString'][0]

In [ ]:
sample_sub2['PredictionString'] = [x.lower() for x in sample_sub2['PredictionString'].unique()]

In [ ]:
#my_list = ["cohort study [lbc1936; n","lothian birth cohort study [lbc1936","Information Resource Incorporated (IRI","on@3", "two#", "thre%e"]
removetable = str.maketrans('', '', ',@#%([')
sample_sub2['PredictionString'] = [s.translate(removetable) for s in sample_sub2['PredictionString']]
print(sample_sub2['PredictionString'])

In [ ]:
stop_words = ["the "
             ]

pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
sample_sub2['PredictionString'] = sample_sub2['PredictionString'].str.replace(pat, '')


#top_words = set(stop_words)
#f = lambda x: ' '.join(w for w in x.split() if not w in stop_words)
#newdata['Verbatim2'] = newdata['Verbatim'].apply(f)

print (sample_sub2['PredictionString'])

In [ ]:
submission2 = pd.DataFrame()
submission2['Id'] = sample_sub2['Id']
submission2['PredictionString'] = sample_sub2['PredictionString']
submission2.head()

<p style="color:blue;font-size:20px;">📝Merging the two DataTables into one Submission File</p>

In [ ]:
##submission = pd.DataFrame(data= {'Id' : sample_sub['Id'], 'PredictionString' : ['','','','']})

In [ ]:
##submission.head()

In [ ]:
merged = pd.concat([submission1, submission2], sort=True)
submission = merged.dropna().groupby(['Id'], as_index=False).agg({'PredictionString' : '|'.join}).reset_index(drop=True)
print(submission)

In [ ]:
(pd.concat(submission.melt(id_vars='Id').dropna() for df in [submission1,submission2])
    .groupby(['Id','variable'])['value'].apply(lambda x: '|'.join(x.unique()))
    .unstack()
)

# **Sorting in a alphabetical Order**

In [ ]:
submission['PredictionString'] = submission['PredictionString'].apply(lambda x: '|'.join(sorted(x.split('|'))))
print(submission)

In [ ]:
submission['PredictionString'][1]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
#test = pd.concat([train_df,train_df],axis=0,ignore_index=True)

<p style="color:green;font-size:20px;">Hope you like this kernel.
    </br>Please don't forget to upvote and leave your valuable comment.   
       </br> Thank you!</p>